[Reference](https://medium.com/@felixpratama242/etl-using-python-postgresql-and-docker-8724d1efbc97)

In [1]:
#import libraries
import pandas as pd

#read dataset:
def extract_data(file):
    data = pd.read_csv(file)

    return data

In [2]:
# from extract import extract_data
import pandas as pd

def transform_data(data):
    # remove unnecessary column
    data.drop(data.iloc[:, 2:44], inplace=True, axis=1)

    # remove missing values:
    data_clean = data.dropna()

    #round consumption values to have only 2 number after comma
    data_column_name = data_clean.columns[-1]
    consumption_data = round(data_clean[data_column_name], 2)

    #create new dictionary for new last column
    new_consumption_data = {
        data_column_name : consumption_data
    }

    #change the new dictionary into new dataframe
    new_consumption_dataframe = pd.DataFrame(new_consumption_data)
    #drop the old last column in dataframe
    data_clean= data_clean.drop([data_column_name], axis = 1)

    #join new last column that has been transformed into dataframe
    data_new = data_clean.join(new_consumption_dataframe)

    return data_new

In [4]:
# from extract import extract_data
# from transform import transform_data
import psycopg2
import argparse

def load_data(file_path, database, host, user, password, port):

    connection = psycopg2.connect(database=database,
                        host=host,
                        user=user,
                        password=password,
                        port=port)

    cursor = connection.cursor()

    print("loading data...")
    data = extract_data(file_path)

    print("transforming data...")
    data_transform = transform_data(data)

    column_name = data_transform.columns[-1]

    #create table
    query_create_table = f"CREATE TABLE IF NOT EXISTS {column_name}(\
    ID SERIAL PRIMARY KEY,\
    continent varchar(50) NOT NULL,\
    country varchar(50) NOT NULL,\
    {column_name} decimal\
    );"

    cursor.execute(query_create_table)

    #start loading data
    print('loading data...')
    for index, row in data_transform.iterrows():
        query_insert_value = f"INSERT INTO {column_name} (continent, country, {column_name}) VALUES ('{row[0]}', \
            '{row[1]}', {row[2]})"

        cursor.execute(query_insert_value)
    connection.commit()

    cursor.close()
    connection.close()

    print("etl success...\n")

    return "all processes completed"

if __name__ == "__main__":

    # Initialize parser
    parser = argparse.ArgumentParser()

    # Adding optional argument
    parser.add_argument("-f", "--file", help = "file path of your dataset")
    parser.add_argument("-db", "--database", help = "database name")
    parser.add_argument("-hs", "--host", help = "your postgresql host")
    parser.add_argument("-u", "--user", help = "postgresql username")
    parser.add_argument("-pass", "--password", help = "postgresql password")
    parser.add_argument("-p", "--port", help = "postgresql port")

    # Read arguments from command line
    args = parser.parse_args()

    load_data(args.file, args.database, args.host, args.user, args.password, args.port)